# In this notebook code for image preprocessing, raining and testing(image by image and overall accuracy) of UE4 image will be developed

In [2]:
import numpy as np
from skimage import io, color, exposure, transform
from skimage.color import rgb2gray
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split  #it came from update scikit learn. https://stackoverflow.com/questions/40704484/importerror-no-module-named-model-selection
import os
import glob
import h5py
import keras

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D,Conv1D
from keras.layers.pooling import MaxPooling2D

from keras.layers.convolutional import Convolution2D, MaxPooling2D

from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras import backend as K
K.set_image_data_format('channels_first')

from matplotlib import pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from matplotlib.pylab import rcParams

#import keras

NUM_CLASSES = 2 # change it with respect to the desired class
IMG_SIZE = 48 # change it if it desired
IMG_depth = 3 # for RGB 3, for B&W it will be 1

# Image preprocessing function

In [3]:
def preprocess_img(img):
    # Histogram normalization in y
    hsv = color.rgb2hsv(img)
    hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
    img = color.hsv2rgb(hsv)

    # central scrop
    min_side = min(img.shape[:-1])
    centre = img.shape[0]//2, img.shape[1]//2
    img = img[centre[0]-min_side//2:centre[0]+min_side//2,centre[1]-min_side//2:centre[1]+min_side//2,:]
#    img = rgb2gray(img)

    # rescale to standard size
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE))

    # roll color axis to axis 0
    img = np.rollaxis(img,-1)

    return img

def get_class(img_path):
    return int(img_path.split('/')[-2])
#     return str(img_path.split('/')[-2]) # returning the folder name. If use -1 that means image name. consider the img_path.

In [4]:
imgs = []
labels = []
root_dir = '/home/atif/machine_learning_stuff/ml_image/copy_image/'
#path='/home/atif/training_by_several_learning_process/flower_photos/00000/'

#all_img_paths = glob.glob(path+ '5547758_eea9edfd54_n_000.jpg')

all_img_paths = glob.glob(os.path.join(root_dir, '*/*.png')) #I have done the training with .png format image. If another type of image will come 
                                                                                    #them .png will be changed by that extension
np.random.shuffle(all_img_paths)
for img_path in all_img_paths:
    try:
        img = preprocess_img(io.imread(img_path))
        label = get_class(img_path)
        imgs.append(img)
        labels.append(label)

        if len(imgs)%1200 == 0: print("Processed {}/{}".format(len(imgs), len(all_img_paths)))
            #print("get it 2")
    except (IOError, OSError):
        print('missed', img_path)
        pass

Processed 1200/7200
Processed 2400/7200
Processed 3600/7200
Processed 4800/7200
Processed 6000/7200
Processed 7200/7200


In [5]:
X = np.array(imgs, dtype='float32') #Keeping the image as an array
X = X.reshape(len(imgs),IMG_depth,IMG_SIZE,IMG_SIZE) # write (IMG_SIZE,IMG_SIZE,1 if you want channel last; 1= grayscale;3=RGB)
# Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]
Y = keras.utils.to_categorical(labels, NUM_CLASSES)

print('X shape: ', X.shape,' type: ',type(X))
print('Y shape: ', Y.shape,' type: ',type(Y))

X shape:  (7200, 3, 48, 48)  type:  <class 'numpy.ndarray'>
Y shape:  (7200, 2)  type:  <class 'numpy.ndarray'>


# Model declaration

In [7]:
def cnn_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=(3, IMG_SIZE, IMG_SIZE),
                     activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same',
                     activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same',
                     activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model

model = cnn_model()

lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
          optimizer=sgd,
          metrics=['accuracy'])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 48, 48)        896       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 46, 46)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 23, 23)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 23, 23)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 23, 23)        18496     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 21, 21)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 10, 10)        0         
__________

In [9]:
path = '/home/atif/machine_learning_stuff/model_file_keras/'

In [11]:
def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

batch_size = 32
epochs = 1
do_train_model=model.fit(X, Y,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,verbose=2,
          #np.resize(img, (-1, <image shape>)
          callbacks=[LearningRateScheduler(lr_schedule),ModelCheckpoint(path+'19_aug_ep_1_label_string.h5', save_best_only=True)])

W0819 23:48:05.314949 140606987949888 deprecation.py:323] From /home/atif/iai_ml_venv/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0819 23:48:05.529354 140606987949888 variables.py:2445] Variable *= will be deprecated. Use `var.assign(var * other)` if you want assignment to the variable value or `x = x * y` if you want a new python Tensor object.
W0819 23:48:05.530908 140606987949888 deprecation.py:506] From /home/atif/iai_ml_venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:680: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the construct

Train on 5760 samples, validate on 1440 samples
Epoch 1/1
 - 26s - loss: 0.2793 - acc: 0.8710 - val_loss: 0.0164 - val_acc: 0.9944


# load saved model

In [29]:
from keras.models import load_model
model = load_model('/home/atif/machine_learning_stuff/model_file_keras/08_aug_ep_100.h5')

# Check total accuracy of teh model using all test images

In [27]:
import pandas as pd
test = pd.read_csv('/home/atif/machine_learning_stuff/ml_image/test_file.csv',sep=';')
# test_image_path =  '/home/atif/machine_learning_stuff/ml_image/test_image_crop/'

X_test = []
y_test = []
i = 0
for file_name, class_id  in zip(list(test['Filename']), list(test['ClassId'])):
    print('filename: ',file_name)
    print('classId: ',class_id)
    img_path = os.path.join('/home/atif/machine_learning_stuff/ml_image/test_image_crop/',file_name)
#     img_path = glob.glob(os.path.join(test_image_path, '*/*.jpg'))
    X_test.append(preprocess_img(io.imread(img_path)))
    y_test.append(class_id)
    
X_test = np.array(X_test)
y_test = np.array(y_test)

filename:  CalgonitFinishKlarspueler.jpg
classId:  0
filename:  crop_CalgonitFinishKlarspuler.jpg
classId:  0
filename:  crop_clgo_finish_klarspueler.jpg
classId:  0
filename:  DenkMitEntkalker.jpg
classId:  7
filename:  1_SM_SomatClassic_53_0_90_60_lit_cropped.jpg
classId:  15


In [28]:
print(img_path)

/home/atif/machine_learning_stuff/ml_image/test_image_crop/1_SM_SomatClassic_53_0_90_60_lit_cropped.jpg


In [26]:
y_test

array([0, 0, 0, 7])

In [30]:
y_pred = model.predict_classes(X_test)
acc = np.sum(y_pred==y_test)/np.size(y_pred)
print("Test accuracy = {}".format(acc))

Test accuracy = 0.2


# check class image by image

In [31]:
import glob

test_image_path = r'/home/atif/machine_learning_stuff/ml_image/test_image_crop/'

my_name = ['SM_CalgonitFinish_2','SM_CalgonitFinishKlarspueler_5','SM_CalgonitFinishMaschinenpfleger_8','SM_CalgonitFinishSpezialSalz_11',
           'SM_CalgonitFinishVorratspack_14','SM_DenkMitEdelstahlreiniger_17','SM_DenkMitEdelstahlReinigerSpray_20','SM_DenkMitEntkalker_23',
           'SM_DenkMitGeschirrReiniger_26','SM_DenkMitGeschirrReinigerClassic_29','SM_DenkMitGeschirrReinigerEvo_32','SM_DenkMitGeschirrReinigerNature_35',
           'SM_DenkMitHygieneAllzweckreiniger_38','SM_DenkMitMaschinenpfleger_41','SM_DenkMitSpezialsalz_44','SM_SomatClassic_53']

img_path = glob.glob(test_image_path+ '/*.jpg')
for image in img_path:
    X_test=[]
    X_test.append(preprocess_img(io.imread(image)))
    X_test = np.array(X_test)
#     plt.imshow(X_test)
    X_test = X_test.reshape(len(X_test),3,IMG_SIZE,IMG_SIZE)
    
    print('image name is: ',image)
    predicted_class = model.predict_classes(X_test)
    print("predicted class: ",predicted_class)
#     print('type-of predicted calss: ', type(predicted_class))
    print('class name is: ',my_name[predicted_class[0]])
    
    probability = model.predict_proba(X_test)
    print("probability: ",probability)
    print('\n')

image name is:  /home/atif/machine_learning_stuff/ml_image/test_image_crop/crop_clgo_finish_klarspueler.jpg
predicted class:  [1]
class name is:  SM_CalgonitFinishKlarspueler_5
probability:  [[2.7340452e-09 9.9999988e-01 1.2844298e-07 3.4400390e-08 4.4906785e-09
  2.9537511e-16 7.6701406e-10 2.5724523e-09 3.7113938e-14 3.9326677e-11
  1.8616154e-15 3.5040252e-14 4.2083198e-18 2.3733964e-16 2.0021346e-11
  3.3514354e-11]]


image name is:  /home/atif/machine_learning_stuff/ml_image/test_image_crop/DenkMitEntkalker.jpg
predicted class:  [1]
class name is:  SM_CalgonitFinishKlarspueler_5
probability:  [[2.9781559e-17 5.0773346e-01 1.0940838e-14 3.1601067e-13 1.0185801e-09
  4.0891950e-16 3.0140297e-11 3.9295414e-01 1.3973983e-12 1.9122426e-06
  2.7194173e-20 3.5659715e-09 3.8106417e-19 2.5966653e-15 2.1486841e-10
  9.9310413e-02]]


image name is:  /home/atif/machine_learning_stuff/ml_image/test_image_crop/CalgonitFinishKlarspueler.jpg
predicted class:  [3]
class name is:  SM_CalgonitFini